## Supervised models
This notebook is intended for giving an introduction the ML supervised models that can be used for Covid detection.

For this notebook to find the new modules created for this project, we need to set its path to be in the root directory.

In [1]:
# Auto reload modules
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

<img src="../images/Supervised_Models.png" width="800"/>

## Loading packages and dependencies

In [3]:
import numpy as np

from src.features.extract_features import load_extracted_features
from sklearn.model_selection import train_test_split
from src.models.build_model import train_advanced_supervised_model, evaluate_model


# Path to the raw data and preprocessed data
raw_data_dir = '../data/raw/COVID-19_Radiography_Dataset/'
IMG_SIZE = 299  # Resize images to IMG_SIZExIMG_SIZE pixels

## Extracting features from images

In [4]:
_, y_normal, X_normal = load_extracted_features(images_dir=raw_data_dir+'{}/images',
                                                    category='NORMAL', dataset_label=0, 
                                                    image_size=IMG_SIZE, image_resized=True, augmentor=True)
_, y_covid, X_covid = load_extracted_features(images_dir=raw_data_dir+'{}/images',
                                                    category='COVID', dataset_label=1, random_seed=42, samples=6576, 
                                                    image_size=IMG_SIZE, image_resized=True, augmentor=True)
_, y_pneumonia, X_pneumonia = load_extracted_features(images_dir=raw_data_dir+'{}/images',
                                                    category='Viral Pneumonia', dataset_label=2, random_seed=42, samples=8847, 
                                                    image_size=IMG_SIZE, image_resized=True, augmentor=True)
_, y_opacity, X_opacity = load_extracted_features(images_dir=raw_data_dir+'{}/images',
                                                    category='Lung_Opacity', dataset_label=3, random_seed=42, samples=4180,
                                                    image_size=IMG_SIZE, image_resized=True, augmentor=True) 

Loaded images for NORMAL: 10192 resized images, 10192 features, and 10192 labels.
Loaded images for COVID: 10192 resized images, 10192 features, and 10192 labels.
Loaded images for Viral Pneumonia: 10192 resized images, 10192 features, and 10192 labels.
Loaded images for Lung_Opacity: 10192 resized images, 10192 features, and 10192 labels.


## Normalizing features

In [5]:
# Combine datasets
X = np.vstack((X_normal, X_covid, X_pneumonia, X_opacity))
y = np.concatenate((y_normal, y_covid, y_pneumonia, y_opacity))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (32614, 299, 299), y_train shape: (32614,)
X_test shape: (8154, 299, 299), y_test shape: (8154,)


In [6]:
# Reshape image data for CNN
X_train = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_test = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (32614, 299, 299, 1)
X_test shape: (8154, 299, 299, 1)


## Training and evaluating models

### Convolutional Neural Networks (CNN)

✅ Strengths:
* Highly accurate for image tasks.
* Learns complex patterns automatically.
* Works well with large image datasets.

❌ Weaknesses:
* Computationally expensive (needs GPUs).
* Requires large labeled datasets.
* Not easily interpretable.

In [7]:
# Train the model
model, history = train_advanced_supervised_model(X_train, y_train, IMG_SIZE, 50, model_type='CNN_Multi')

2025-03-08 17:04:26.455327: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2025-03-08 17:04:26.455366: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 128.00 GB
2025-03-08 17:04:26.455389: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 48.00 GB
2025-03-08 17:04:26.455407: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-08 17:04:26.455423: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 299, 299, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ (None, 256, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 256, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 524288)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    67,108,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 67,109,953 (256.00 MB)

 Trainable params: 67,109,953 (256.00 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50


/Users/rehabaam/codes/python/datascientest/project/ds_covid19_project/.ds_covid19/lib/python3.12/site-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/Users/rehabaam/codes/python/datascientest/project/ds_covid19_project/.ds_covid19/lib/python3.12/site-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)
2025-03-08 17:04:44.823792: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


816/816 ━━━━━━━━━━━━━━━━━━━━ 52s 62ms/step - accuracy: 0.2505 - loss: 0.0000e+00 - val_accuracy: 0.2470 - val_loss: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/50
816/816 ━━━━━━━━━━━━━━━━━━━━ 50s 61ms/step - accuracy: 0.2523 - loss: 0.0000e+00 - val_accuracy: 0.2470 - val_loss: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/50
816/816 ━━━━━━━━━━━━━━━━━━━━ 50s 61ms/step - accuracy: 0.2502 - loss: 0.0000e+00 - val_accuracy: 0.2470 - val_loss: 0.0000e+00 - learning_rate: 0.0010
Epoch 4/50
816/816 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.2480 - loss: 0.0000e+00
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
816/816 ━━━━━━━━━━━━━━━━━━━━ 50s 61ms/step - accuracy: 0.2480 - loss: 0.0000e+00 - val_accuracy: 0.2470 - val_loss: 0.0000e+00 - learning_rate: 0.0010
Epoch 5/50
816/816 ━━━━━━━━━━━━━━━━━━━━ 50s 62ms/step - accuracy: 0.2495 - loss: 0.0000e+00 - val_accuracy: 0.2470 - val_loss: 0.0000e+00 - learning_rate: 5.0000e-04
Epoch 6/50
816/816 ━━━━━━━━━━━━━━━━━━━━ 5

In [8]:
train_loss, train_acc = history.history['loss'][-1], history.history['accuracy'][-1]
print(f"Train Accuracy: {train_acc:.4f}, Train Loss: {train_loss:.4f}")

test_loss, test_acc = evaluate_model("Binary classification [Normal, Others] for images without masks", model, X_test, y_test, model_type="CNN")
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")


Train Accuracy: 0.2508, Train Loss: 0.0000
255/255 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.2564 - loss: 0.0000e+00
  8/255 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step 

/Users/rehabaam/codes/python/datascientest/project/ds_covid19_project/.ds_covid19/lib/python3.12/site-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


255/255 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step


/Users/rehabaam/codes/python/datascientest/project/ds_covid19_project/.ds_covid19/lib/python3.12/site-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
Successfully registered model 'tensorflow-CNN-2025-03-08'.
2025/03/08 17:10:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tensorflow-CNN-2025-03-08, version 1


🏃 View run CNN-2025-03-08 17:10:02.333245 at: http://localhost:8080/#/experiments/877999877627272198/runs/52ad6bee3bcd4478828fc71b066841c1
🧪 View experiment at: http://localhost:8080/#/experiments/877999877627272198
Test Accuracy: 0.2499, Test Loss: 0.0000


Created version '1' of model 'tensorflow-CNN-2025-03-08'.


### Transfer learning

✅ Strengths
* Transfer learning reduces training time while maintaining high accuracy.
* Fine-tuning improves performance when sufficient data is available.
* Combining deep features with statistical features can enhance results.

In [9]:
model, history = train_advanced_supervised_model(X_train, y_train, IMG_SIZE, 50, model_type="Transfer Learning Multi")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 299, 299,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 299, 299,  │          7 │ rescaling_1[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 299, 299,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 301, 301,  │          0 │ rescaling_2[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 150, 150,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 150, 150,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 150, 150,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 150, 150,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 150, 150,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 150, 150,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 150, 150,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 150, 150,  │        512 │ block1a_se_excit

 Total params: 4,213,668 (16.07 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/50


/Users/rehabaam/codes/python/datascientest/project/ds_covid19_project/.ds_covid19/lib/python3.12/site-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


816/816 ━━━━━━━━━━━━━━━━━━━━ 94s 111ms/step - accuracy: 0.2503 - loss: 0.0000e+00 - val_accuracy: 0.2470 - val_loss: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 2/50
816/816 ━━━━━━━━━━━━━━━━━━━━ 79s 97ms/step - accuracy: 0.2466 - loss: 0.0000e+00 - val_accuracy: 0.2470 - val_loss: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 3/50
816/816 ━━━━━━━━━━━━━━━━━━━━ 80s 98ms/step - accuracy: 0.2456 - loss: 0.0000e+00 - val_accuracy: 0.2470 - val_loss: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 4/50
816/816 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.2489 - loss: 0.0000e+00
Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
816/816 ━━━━━━━━━━━━━━━━━━━━ 84s 103ms/step - accuracy: 0.2489 - loss: 0.0000e+00 - val_accuracy: 0.2470 - val_loss: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 5/50
816/816 ━━━━━━━━━━━━━━━━━━━━ 96s 117ms/step - accuracy: 0.2550 - loss: 0.0000e+00 - val_accuracy: 0.2470 - val_loss: 0.0000e+00 - learning_rate: 5.0000e-05
Epoch 6/50
816/816 ━━━

In [10]:
train_loss, train_acc = history.history['loss'][-1], history.history['accuracy'][-1]
print(f"Train Accuracy: {train_acc:.4f}, Train Loss: {train_loss:.4f}")

test_loss, test_acc = evaluate_model("Binary classification [Normal, Others] for images without masks",model, X_test, y_test, model_type="Transfer Learning")
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

Train Accuracy: 0.2508, Train Loss: 0.0000
255/255 ━━━━━━━━━━━━━━━━━━━━ 23s 91ms/step - accuracy: 0.2564 - loss: 0.0000e+00
255/255 ━━━━━━━━━━━━━━━━━━━━ 24s 90ms/step


Successfully registered model 'tensorflow-Transfer Learning-2025-03-08'.
2025/03/08 17:20:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tensorflow-Transfer Learning-2025-03-08, version 1


🏃 View run Transfer Learning-2025-03-08 17:19:43.408257 at: http://localhost:8080/#/experiments/877999877627272198/runs/2718a22fb9af47e0a7af333ddd277236
🧪 View experiment at: http://localhost:8080/#/experiments/877999877627272198
Test Accuracy: 0.2499, Test Loss: 0.0000


Created version '1' of model 'tensorflow-Transfer Learning-2025-03-08'.
